In [1]:
import pandas as pd
from datetime import datetime
import time

starttime = time.time()

#remove columns view max limit
pd.set_option('display.max_columns', None)

loan_lenders = pd.read_csv("loans_lenders.csv")
#parsing to datetime
loan = pd.read_csv("loans.csv",parse_dates=['planned_expiration_time','disburse_time'])
lenders=pd.read_csv('lenders.csv')
country_stats=pd.read_csv('country_stats.csv')

importtime = time.time()


<h3>Exercise 1<h3>

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [2]:
display(loan_lenders)

loan_lenders_list = loan_lenders.values.tolist()
loan_id=[]
lenders_list=[]

for row in loan_lenders_list:
    names_splitted=row[1].split(',')
    id_=row[0]
    for name in names_splitted:
        lenders_list.append(name)
        loan_id.append(id_)
    
loan_lenders_normalized = pd.DataFrame({"loan_id": loan_id, "lender": lenders_list}, columns=["loan_id", "lender"])
display(loan_lenders_normalized)

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


<h3>Exercise 2<h3>
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. 
If any of those two dates is missing, also the duration must be missing.

In [3]:
#dropping rows where any of the two dates is missing
loan_duration=loan.dropna(subset=['planned_expiration_time'])
loan_duration=loan_duration.dropna(subset=['disburse_time'])

loan_duration['duration']=loan['planned_expiration_time']-loan['disburse_time']
loan_duration['duration']=loan_duration['duration'].dt.days
display(loan_duration[['loan_id','disburse_time','planned_expiration_time','duration']])

,loan_id,disburse_time,planned_expiration_time,duration
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57
...,...,...,...,...
1419602,988180,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,39
1419603,988213,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,39
1419604,989109,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,51
1419605,989143,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,63


<h3>Exercise 3<h3>
Find the lenders that have funded at least twice.

In [4]:
#grouping by lender name
loan_lenders_grouped = loan_lenders_normalized.groupby('lender').count()
loan_lenders_grouped=loan_lenders_grouped.rename(columns={"loan_id":"nr_funds"})


loan_lenders_grouped.loc[loan_lenders_grouped['nr_funds'] >= 2]

,nr_funds
lender,
000,39
00000,39
0002,70
0101craign0101,71
0132575,4
...,...
zyrorl,3
zzaman,11
zzanita,2


<h3>Exercise 4<h3>
For each country, compute how many loans have involved that country as borrowers.

In [5]:
#counting unique values
nr_loans = loan['country_name'].value_counts().reset_index(name='loans')
nr_loans

,index,loans
0,Philippines,285336
1,Kenya,143699
2,Peru,86000
3,Cambodia,79701
4,El Salvador,64037
...,...,...
91,Papua New Guinea,1
92,Mauritania,1
93,Botswana,1
94,Uruguay,1


<h3>Exercise 5<h3>
For each country, compute the overall amount of money borrowed.

In [6]:
country_loans = loan.groupby('country_name')['loan_amount'].sum().reset_index(name='overall_money_borrowed')
country_loans

,country_name,overall_money_borrowed
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0
...,...,...
91,Vietnam,24681100.0
92,Virgin Islands,10000.0
93,Yemen,3444000.0
94,Zambia,1978975.0


<h3>Exercise 6<h3>
Like the previous point, but expressed as a percentage of the overall amount lent.

In [7]:
total_money=loan['loan_amount'].sum()
country_loans['overall_money_perc'] = country_loans['overall_money_borrowed']/total_money*100
country_loans

,country_name,overall_money_borrowed,overall_money_perc
0,Afghanistan,1967950.0,0.166573
1,Albania,4307350.0,0.364586
2,Armenia,22950475.0,1.942589
3,Azerbaijan,14784625.0,1.251410
4,Belize,150175.0,0.012711
...,...,...,...
91,Vietnam,24681100.0,2.089074
92,Virgin Islands,10000.0,0.000846
93,Yemen,3444000.0,0.291509
94,Zambia,1978975.0,0.167506


<h3>Exercise 6<h3>
Like the three previous points, but split for each year (with respect to disburse time).

Exercise 4v2 ---> how many loans have involved that country as borrowers.

In [8]:
loan['year']=loan['disburse_time'].dt.year
nr_loans_annual=loan.groupby(['year','country_name']).size().reset_index(name='loans')
nr_loans_annual

,year,country_name,loans
0,2005.0,Bulgaria,3
1,2005.0,Cambodia,2
2,2005.0,Ecuador,10
3,2005.0,Gaza,8
4,2005.0,Honduras,72
...,...,...,...
743,2018.0,Tanzania,45
744,2018.0,Thailand,13
745,2018.0,Uganda,2
746,2018.0,Uruguay,1


<h3>Exercise 5v2 ---> overall amount of money borrowed.<h3>

In [9]:
country_loans_annual=loan.groupby(['country_name','year'])['loan_amount'].sum().reset_index()
country_loans_annual

,country_name,year,loan_amount
0,Afghanistan,2007.0,194975.0
1,Afghanistan,2008.0,365375.0
2,Afghanistan,2009.0,585125.0
3,Afghanistan,2010.0,563350.0
4,Afghanistan,2011.0,245125.0
...,...,...,...
743,Zimbabwe,2013.0,678525.0
744,Zimbabwe,2014.0,1311575.0
745,Zimbabwe,2015.0,723625.0
746,Zimbabwe,2016.0,788600.0


<h3>Exercise 6v2 --->  overall amount of money borrowed expressed as percentage

In [10]:
percentage_annual=loan.groupby(['country_name','year'])['loan_amount'].sum()/loan.groupby('year')['loan_amount'].sum()
percentage_annual.reset_index()

,country_name,year,loan_amount
0,Afghanistan,2007.0,0.012623
1,Afghanistan,2008.0,0.009268
2,Afghanistan,2009.0,0.009803
3,Afghanistan,2010.0,0.007759
4,Afghanistan,2011.0,0.002616
...,...,...,...
743,Zimbabwe,2013.0,0.005139
744,Zimbabwe,2014.0,0.008613
745,Zimbabwe,2015.0,0.004644
746,Zimbabwe,2016.0,0.004899


<h3>Exercise 8<h3>
For each lender, compute the overall amount of money lent. 
For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [11]:
#counting how many lenders partacipate to total amount
loan_merged=pd.merge(loan[['loan_id','loan_amount']],loan_lenders_normalized,on='loan_id')
count_shares=loan_merged.groupby('loan_id').size().reset_index(name='counts')
lenders_shares=pd.merge(count_shares,loan_merged,on='loan_id')
#dividing for every lender that contributed
lenders_shares['single_share']=lenders_shares['loan_amount']/lenders_shares['counts']
display(lenders_shares)
lenders_overall_money=lenders_shares.groupby('lender').sum()['single_share'].reset_index(name='overall_amount')
display(lenders_overall_money)


,loan_id,counts,loan_amount,lender,single_share
0,84,3,500.0,ward,166.666667
1,84,3,500.0,michael,166.666667
2,84,3,500.0,brooke,166.666667
3,85,2,500.0,michael,250.000000
4,85,2,500.0,patrick,250.000000
...,...,...,...,...,...
28293907,1444051,1,225.0,lauren1424,225.000000
28293908,1444053,1,725.0,dario6858,725.000000
28293909,1444058,1,25.0,el5018,25.000000
28293910,1444063,1,50.0,pgs67,50.000000


,lender,overall_amount
0,000,1672.618411
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,2287.291955


<h3>Exercise 9<h3>
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. 
Since the country of the lender is often unknown, 
you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [12]:
#suppressing scientific notation
pd.options.display.float_format = '{:.2f}'.format

#dropping rows with missing values
lenders_country=lenders.dropna(subset=['country_code'])

#adding country code to previous df with overall money
lenders_country_money=pd.merge(lenders_country[['permanent_name','country_code']],lenders_overall_money,left_on='permanent_name',right_on='lender')
lenders_country_money=lenders_country_money.drop('permanent_name',axis=1)
loans_per_country=lenders_country_money.groupby('country_code').sum()['overall_amount'].reset_index()
borrows_per_country=loan.groupby('country_code').sum()['loan_amount'].reset_index()
overall_country=pd.merge(loans_per_country,borrows_per_country,how='outer')

#considering NA as zeros (cases where a country has only loans or only borrows)
overall_country=overall_country.fillna(0)
overall_country['difference_amount']=overall_country['overall_amount']-overall_country['loan_amount']
overall_country.columns = ['country_code','overall_lent','overall_borrowed','difference_amount']
display(overall_country)

,country_code,overall_lent,overall_borrowed,difference_amount
0,AD,115.15,0.00,115.15
1,AE,116011.09,0.00,116011.09
2,AF,6260.80,1967950.00,-1961689.20
3,AL,732.29,4307350.00,-4306617.71
4,AM,1042.01,22950475.00,-22949432.99
...,...,...,...,...
202,SB,0.00,815575.00,-815575.00
203,SO,0.00,308725.00,-308725.00
204,TD,0.00,20075.00,-20075.00
205,WS,0.00,12212175.00,-12212175.00


<h3>Exercise 10<h3>
Which country has the highest ratio between the difference computed at the previous point and the population?

In [13]:
#merging output from ex9 with country_stats
ratio_country=pd.merge(overall_country[['country_code','difference_amount']],country_stats[['country_code','country_name','population']],how='inner')
ratio_country['ratio']=ratio_country['difference_amount']/ratio_country['population']
#searching for highest ratio
ratio_country.loc[ratio_country['ratio'].idxmax()]

country_code                 LU
difference_amount     135863.29
country_name         Luxembourg
population               583455
ratio                      0.23
Name: 82, dtype: object

<h3>Exercise 11<h3>
Which country has the highest ratio between the difference computed at point 9 
and the population that is not below the poverty line?

In [14]:
poverty_per_country=pd.merge(overall_country[['country_code','difference_amount']],country_stats[['country_code','country_name','population','population_below_poverty_line']],how='inner')
# calculating population over poverty line:  tot_pop*perc_over_povl/100
poverty_per_country['pop_over_povl']=(poverty_per_country['population']*(100-poverty_per_country['population_below_poverty_line']))//100
poverty_per_country['ratio']=poverty_per_country['difference_amount']/poverty_per_country['pop_over_povl']
#searching for highest ratio
poverty_per_country.loc[poverty_per_country['ratio'].idxmax()]

country_code                             CA
difference_amount                4480813.77
country_name                         Canada
population                         36624199
population_below_poverty_line          9.40
pop_over_povl                   33181524.00
ratio                                  0.14
Name: 21, dtype: object

<h3>Exercise 12<h3>
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 
For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, 
and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 
5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [15]:
#returns a list with every amount per year
def divide_money_by_year(row):
    #localize required for subtraction of two timestamps
    start= row['disburse_time'].tz_localize(None)
    end= row['planned_expiration_time'].tz_localize(None)
    money= row['loan_amount']
    years_difference = end.year - start.year 
    days=[]

    #variable keeping track of latest year added to the list
    index = start
    while(years_difference >= 0):
        #case where disburse and expiration is in same year
        if years_difference == 0:
            days.append((end - index).days)
            #last year, exit from while loop
            years_difference = -1
        #case of multiple years    
        else:
            last_of_the_year = datetime(index.year+1,1,1)
            days.append((last_of_the_year - index).days)
            index = last_of_the_year
            years_difference = years_difference - 1
   
    if sum(days)==0:
        #creating a list for returning same type
        moneylist=[]
        moneylist.append(money)
        return moneylist
    else:
        for i in range(len(days)):
            #calculating amount per year
            days[i] = days[i]*money/sum(days)
    return(days)        


In [16]:
lender_per_year = loan
lender_per_year=lender_per_year.dropna(subset=['planned_expiration_time'])
lender_per_year=lender_per_year.dropna(subset=['disburse_time'])
#applying function to loan df
lender_per_year['money_per_year'] = lender_per_year.apply(divide_money_by_year, axis=1)
lender_per_year[['loan_name','disburse_time','planned_expiration_time','money_per_year']]

,loan_name,disburse_time,planned_expiration_time,money_per_year
0,Aivy,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,"[21.22641509433962, 84.3216661845531]"
1,Idalia Marizza,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,"[46.31578947368421, 257.8352180936995]"
2,Aasia,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,[400.0]
3,Gulmira,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,[625.0]
4,Ricky\t,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,"[104.3859649122807, 123.99416736102845]"
...,...,...,...,...
1419602,NaN,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,"[389.7435897435897, 1.0236892184526545]"
1419603,Perlita,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,"[292.10526315789474, 1.0235230741605315]"
1419604,Okyeso Nyame Group,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,"[2328.0, 2.0815450643776825]"
1419605,Exequila,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,"[93.54838709677419, 4.1005291005291005]"


In [17]:
lender_per_year['diff']=lender_per_year['planned_expiration_time'].dt.year-lender_per_year['disburse_time'].dt.year
lender_per_year.loc[lender_per_year['diff'].idxmax()]

loan_id                                                                      1077942
loan_name                                                                    Patrick
original_language                                                            English
description                        My name is Patrick Marubu Njoroge. I was born ...
description_translated             My name is Patrick Marubu Njoroge. I was born ...
funded_amount                                                                 650.00
loan_amount                                                                   650.00
status                                                                        funded
activity_name                                                               Services
sector_name                                                                 Services
loan_use                           buy raw materials and a kiln for my cook stove...
country_code                                                     

In [19]:
#summing total amount per year

#storing in a dictionary
years = dict()
    
for i,row in lender_per_year.iterrows():
        year_start= row['disburse_time'].year
        year_finish=row['planned_expiration_time'].year
        if len(row['money_per_year']) == 1:    
            years[year_start] = years.get(year_start,0)+row['money_per_year'][0]
        if len(row['money_per_year']) == 2:
            years[year_start] = years.get(year_start,0)+row['money_per_year'][0]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][1]
        if len(row['money_per_year']) == 3:
            years[year_start] = years.get(year_start,0)+row['money_per_year'][0]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][1]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][2]
        if len(row['money_per_year']) == 4:
            years[year_start] = years.get(year_start,0)+row['money_per_year'][0]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][1]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][2]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][3]
        if len(row['money_per_year']) == 5:
            years[year_start] = years.get(year_start,0)+row['money_per_year'][0]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][1]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][2]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][3]
            years[year_finish] = years.get(year_start,0)+row['money_per_year'][4]

            
years                    

{2013: 2031229.2159526998,
 2014: 3343962.692399011,
 2015: 3600229.0464434223,
 2012: 1010384.8951696253,
 2016: 3600231.091234075,
 2017: 3217805.9695254937,
 2018: 2104434.187236917,
 2011: 639263.9662771509}

In [20]:
endtime = time.time()
print("Executing time: {} minutes".format(int((endtime-starttime)/60)))
print("of which {} seconds were for importing ".format(int((importtime-starttime))))

Executing time: 5 minutes
of which 41 seconds were for importing 
